In [1]:
import cv2 as cv
from numpy.random import default_rng
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
#Descargamos el vídeo desde Github
!wget slow_traffic_small.mp4 https://github.com/juannieu/VPC1/raw/main/TP7/slow_traffic_small.mp4

--2022-05-01 18:05:43--  http://slow_traffic_small.mp4/
Resolving slow_traffic_small.mp4 (slow_traffic_small.mp4)... failed: Name or service not known.
wget: unable to resolve host address ‘slow_traffic_small.mp4’
--2022-05-01 18:05:43--  https://github.com/juannieu/VPC1/raw/main/TP7/slow_traffic_small.mp4
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/juannieu/VPC1/main/TP7/slow_traffic_small.mp4 [following]
--2022-05-01 18:05:43--  https://raw.githubusercontent.com/juannieu/VPC1/main/TP7/slow_traffic_small.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2018126 (1.9M) [application/oct

In [3]:
def mostrar(imagen):
  plt.figure()
  plt.imshow(imagen)
  plt.show()

In [10]:
def cuantosFramesTieneElVideo(videoCapturado):
  varFrames=0
  varStop=False
  while varStop==False:
    re,frame=videoCapturado.read()
    varFrames=varFrames+1
    if frame is None:
        varStop=True
        return varFrames-1
        #También pude haber usado #frames2= video.get(cv.CAP_PROP_FRAME_COUNT), pero me di cuenta cuando esto ya estaba

In [5]:
def framesAleatorios(cuentaFrames,N):
  framesObservados=[]
  numerosAleatoriosGen = default_rng()
  print('cuentaFrames esdfgdfg',cuentaFrames)
  print('N edfgfdgs ',N)
  numeros = numerosAleatoriosGen.choice(cuentaFrames, size=N, replace=False)
  numeros = np.sort(numeros)
  #print(np.dtype(numeros))
  return numeros

In [53]:
def imgMediana(N):
  elArr=[]

#Abro el vídeo
  elVideo=cv.VideoCapture('slow_traffic_small.mp4')
#Cuento cuántos frames tengo
  numFrames=cuantosFramesTieneElVideo(elVideo)
#Generamos la lista de frames aleatorios
  framex=framesAleatorios(numFrames,N)

#Leo el vídeo
  elVideo=cv.VideoCapture('slow_traffic_small.mp4')
  exito,cuadro=elVideo.read()
  indice=0
  while exito:
    imagen_Gris=cv.cvtColor(cuadro,cv.COLOR_BGR2GRAY)
    if indice in framex:
    #mostrar(imagen_Gris)
      elArr.append(imagen_Gris)
    exito,cuadro=elVideo.read()
    indice=indice+1
  med=np.median(elArr,axis=0)
  mostrar(med)
  #print(indice)
  return med


In [61]:
def naive():
  #Recibo los parámetros de entrada
  print('Cuántos Frames uso?')
  N=int(input())
  print('Cada cuánto tiempo recalculo?(En segundos)')
  intervalo=int(input())
  tiempo_inicial=time.time()
  tiempo=tiempo_inicial
  #Considerando que el vídeo dura 30 segundos
  while tiempo<tiempo_inicial+30:
    resta=tiempo-tiempo_inicial
    tiempo=time.time()
    if resta%intervalo==0:
      laMediana=imgMediana(N)
    #Leo el vídeo
    elVideoE=cv.VideoCapture('slow_traffic_small.mp4')
    exitoE,cuadroE=elVideoE.read()
    #Resto cada frame a la mediana
    while exitoE:
      imagen_Gris=cv.cvtColor(cuadroE,cv.COLOR_BGR2GRAY)
      exitoE,cuadroE=elVideoE.read()
      menosMediana=imagen_Gris-laMediana
      mostrar(menosMediana)

In [ ]:
for i in range(0,2):
  if i==0:
    naive()
  else:
    #Y luego comparamos con modelos Gaussianos
    backSub = cv.createBackgroundSubtractorKNN()
    backSub = cv.createBackgroundSubtractorKNN(detectShadows = True)

    filename = 'vtest.avi'
    capture = cv.VideoCapture(filename)

    if not capture.isOpened:
        print('Falla al abrir el archivo: ' + filename)
        exit(0)

# Corremos la sustraccion
#------------------------
    while True:
    # Leemos un frame
        ret, frame = capture.read()
        if frame is None:
            break
    
    # Aplicamos la sustracción al frame leído
    #----------------------------------------
    # Cada frame se utiliza tanto para calcular la máscara de primer plano como para actualizar el fondo.
    # Si se desea cambiar la tasa de aprendizaje utilizada para actualizar el modelo de fondo, es posible
    # establecer una tasa de aprendizaje específica pasando un parámetro al método apply.
        fgMask = backSub.apply(frame)
    
    # Escribimos sobre la imagen el número de frame procesado
        cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
        cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
                  cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    
        mostrar(frame)
        mostrar(fgMask)


